In [ ]:
%tensorflow_version 1.x
from keras.datasets import mnist
import matplotlib.pyplot as plt
from keras.layers import Dense, Dropout, Input, BatchNormalization, Flatten, Reshape, ZeroPadding2D, Activation
from keras.models import Sequential, Model
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.optimizers import Adam
import numpy as np
from tqdm import tqdm_notebook
import itertools

TensorFlow 1.x selected.


Using TensorFlow backend.


Based on Ian Goodfellow's Paper GAN is developed. However, DCGAN is advanced version of GAN which is developed here based on Alec Radford, Luke Metz, Soumith Chintala's Paper "UNSUPERVISED REPRESENTATION LEARNING WITH DEEP CONVOLUTIONAL GENERATIVE ADVERSARIAL NETWORKS". But, The idea is very much similar to GOODFELLOW'S GAN Model.

In [ ]:
class DCGAN():

  def __init__(self):
    self.img_rows = 28
    self.img_cols = 28
    self.channels = 1
    self.img_shape = (self.img_rows, self.img_cols, self.channels) #Channels is optional
    self.latent_dim = 100

    optimizer = Adam(0.0002, 0.5)

    self.D = self.create_discriminator()
    self.D.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    self.G = self.create_generator()
    z = Input(shape=(self.latent_dim,))
    counterfiet= self.G(z)
    self.D.trainable=False
    police= self.D(counterfiet)
    self.dcgan=Model(inputs=z, outputs=police)
    self.dcgan.compile(loss='binary_crossentropy', optimizer=optimizer)


  def create_generator(self):
    generator_model=Sequential()

    #Initial Shape 7x7. Two times Upsample will make it 28*28
    
    generator_model.add(Dense(256*7*7, activation = LeakyReLU(0.2), input_dim = self.latent_dim))
    generator_model.add(Reshape((7, 7, 256)))
    generator_model.add(UpSampling2D())

    generator_model.add(Conv2D(128, kernel_size=3, padding="same"))
    generator_model.add(BatchNormalization(momentum=0.8))
    generator_model.add(Activation(LeakyReLU(0.2)))
    generator_model.add(UpSampling2D())

    generator_model.add(Conv2D(64, kernel_size=3, padding="same"))
    generator_model.add(BatchNormalization(momentum=0.8))
    generator_model.add(Activation(LeakyReLU(0.2)))
   
    generator_model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
    generator_model.add(Activation("tanh"))

    noise = Input(shape=(self.latent_dim,))
    gen_img = generator_model(noise)

    return Model(inputs=noise , outputs=gen_img)

  
  def create_discriminator(self):

    discriminator_model = Sequential()

    discriminator_model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
    discriminator_model.add(LeakyReLU(alpha=0.2))
    discriminator_model.add(Dropout(0.25))

    discriminator_model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    discriminator_model.add(ZeroPadding2D(padding=((0,1),(0,1)))) #Pad Bottom and Right Respectively
    discriminator_model.add(BatchNormalization(momentum=0.8))
    discriminator_model.add(LeakyReLU(alpha=0.2))
    discriminator_model.add(Dropout(0.25))

    discriminator_model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    discriminator_model.add(BatchNormalization(momentum=0.8))
    discriminator_model.add(LeakyReLU(alpha=0.2))
    discriminator_model.add(Dropout(0.25))

    discriminator_model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
    discriminator_model.add(BatchNormalization(momentum=0.8))
    discriminator_model.add(LeakyReLU(alpha=0.2))
    discriminator_model.add(Dropout(0.25))

    discriminator_model.add(Flatten())
    discriminator_model.add(Dense(1, activation='sigmoid'))

    img = Input(shape=self.img_shape)
    validity = discriminator_model(img)

    return Model(inputs=img, outputs=validity)

  def training(self, epochs, batch_size=128, sample_interval=10000):

    (X_train , _), (_ , _) = mnist.load_data()
    X_train = X_train/127.5 -1
    X_train = np.expand_dims(X_train, axis=3) #In Order to add the channel Information

    #Truth Value
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in tqdm_notebook(range(epochs+1)):

      idx = np.random.randint(0, X_train.shape[0], batch_size)
      imgs = X_train[idx]
      noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

      self.dcgan.train_on_batch(noise, valid) #Fooling Tricks
      gen_imgs = self.G.predict(noise)

      self.D.train_on_batch(imgs, valid)
      self.D.train_on_batch(gen_imgs, fake)

      if epoch % sample_interval == 0:
        self.sample_images(epoch)

  
  def sample_images(self, epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, self.latent_dim))
    gen_imgs = self.G.predict(noise)

    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axis = plt.subplots(r, c, figsize=(5, 5))

    for m, n in itertools.product(range(5), range(5)):
      axis[m, n].get_xaxis().set_visible(False)
      axis[m, n].get_yaxis().set_visible(False)

    for i in range(25):
      row = i // 5
      col = i % 5
      axis[row, col].cla()    #Clear the current axes.
      axis[row, col].imshow(np.reshape(gen_imgs[i], (28, 28)), cmap='gray')

    plt.savefig('gan_generated_image %d.png' %epoch)
    plt.close()  

In [ ]:
if __name__ == '__main__':
  dcgan = DCGAN()
  dcgan.training(epochs=80000, batch_size=128, sample_interval=10000)